### Defining paths for inputs and output files

In [ ]:
# Import standard utility modules
import os     # For creating and removing a directory, fetching its contents, changing and identifying the current directory, etc.
import shutil # Perform high-level operation like copy and create on files and collections of files.

### Set GIS input data directory

In [ ]:
# Define the directory  
gis_data_folder = "/home/yoni/Documents/GIS_Training"

### Create the input data directory 

In [ ]:
# If you are on the python/juppter notebook dont forget to use the exclamation `!`   
!mkdir -p /home/yoni/Documents/GIS_Training

### Change the directory to the input data and directory and get current working directory

In [ ]:
os.chdir(gis_data_folder)
cwd = os.getcwd()
cwd

### Set path to input data

In [ ]:
data_folder = os.path.join(cwd, 'GIS_DATA')
data_folder

### Set path to output data

In [ ]:
output_folder = os.path.join(cwd, 'Outputs')
output_folder

### Clear all outputs from previous runs and re-creating the output directory

In [ ]:
if os.path.exists(output_folder):
   shutil.rmtree(output_folder)
os.mkdir(output_folder)

### Create domain boundary files

In [ ]:
### Let’s define the path for the geo_em.d01.nc

in_geogrid = os.path.join(data_folder, 'example_data3/geo/geo_em.d03.nc')
in_geogrid

### Run the script with required parameters

In [ ]:
# Print information to screen for reference
print('Command to run:\n')
print('Python Create_Domain_Boundary_Shapefile.py \\\n\t -i {0} \\\n\t -o {1}\n'.format(in_geogrid, output_folder))

! python Create_Domain_Boundary_Shapefile.py -i {in_geogrid} -o {output_folder}

### Visualize the domain boundary shapefile

In [ ]:
import json
import geopandas
import ipyleaflet
from ipyleaflet import Map, GeoJSON, ScaleControl, FullScreenControl, basemaps, SplitMapControl, basemap_to_tiles, LayersControl
from jupyter_functions import create_map

import warnings
warnings.filterwarnings("ignore")

# Setup display items
boundary_shp = os.path.join(output_folder,'geo_em.d03_boundary.shp')
b_shp = geopandas.read_file(boundary_shp)
b_shp = b_shp.to_crs(epsg=4326)

# Export vector to GeoJSON
b_json = os.path.join(output_folder, 'boundary.json')
b_shp.to_file(b_json, driver='GeoJSON')

# Read GeoJSON
with open(b_json, 'r') as f:
    data = json.load(f)

# Obtain vector center point
x = b_shp.geometry.centroid.x
y = b_shp.geometry.centroid.y
map_center = y[0], x[0]

# Instantiate map object
m = Map(center=(41.50, -73.73), zoom=10, scroll_wheel_zoom=True)

# Read GeoJSON
with open(b_json, 'r') as f:
    data = json.load(f)

# Obtain vector center point
x = b_shp.geometry.centroid.x
y = b_shp.geometry.centroid.y
map_center = y[0], x[0]

# Instantiate map object
m = create_map(map_center, 10)

# Read GeoJSON
geo_json = GeoJSON(data=data, name='Domain boundary')

# Define basemaps to swipe between
right_layer = basemap_to_tiles(basemap=basemaps.OpenStreetMap.Mapnik)
left_layer = basemap_to_tiles(basemap=basemaps.Esri.WorldImagery)

# Setup basemap swipe control
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)
m.add_layer(geo_json)

# Draw map
m

### Build GeoTiff raster from a surface elevation

In [ ]:
# Define the variable to export to raster
in_var = "HGT_M"

# Define the output raster file using variable name defined above
out_file = os.path.join(output_folder, f'{in_var}.tif')

# Print information to screen for reference
print('Command to run:\n')
print('python Build_GeoTiff_From_Geogrid_File.py \\\n\t -i {0} \\\n\t -v {1} \\\n\t -o {2}\n'.format(in_geogrid, in_var, out_file))

# Run the script with required parameters
! python Build_GeoTiff_From_Geogrid_File.py -i {in_geogrid} -v {in_var} -o {out_file}

### Draw the basemap

In [ ]:
# Import Python visualization libraries
import rasterio
from matplotlib import pyplot
from osgeo import gdal
from ipyleaflet import ImageOverlay
from jupyter_functions import cmap_options, show_raster_map

# Create a map object from pre-build function
m2 = create_map(map_center, 10)

# Render the map
m2

### Overlay the surface elevation on the basemap

In [ ]:
show_raster_map(out_file, m2, b_shp, output_folder)

### Building the hydrologic routing grids

In [ ]:
import Build_Routing_Stack

# Define script input parameters using python variables
in_geogrid = os.path.join(data_folder, 'example_data3/geo/geo_em.d03.nc')
csv = os.path.join(data_folder, 'example_data3/forecast_points.csv')
in_dem = os.path.join(data_folder, '/home/yoni/Documents/Dechatu/merged_pro.tif')
regrid_factor = 4
routing_cells = 25
out_zip = os.path.join(output_folder, 'Gridded_test.zip')

### Execute the routing scrip

In [ ]:
# Print information to screen for reference
#print('Command to run:\n')
#print('python Build_Routing_Stack.py \\\n\t -i {0} \\\n\t \\\n\t --CSV {2} \\\n\t -d {3} \\\n\t -R {4} \\\n\t -t {5} \\\n\t -o {6}\n'.format(in_geogrid, csv, in_dem, regrid_factor, routing_cells, out_zip))

# Run the script with required parameters
! python Build_Routing_Stack.py -i {in_geogrid} \
--CSV {csv} \
-d {in_dem} \
-R {regrid_factor} \
-t {routing_cells} \
-o {out_zip}

### Define output directory to store GeoTiff output of all routing stack grids

In [ ]:
raster_outputs = os.path.join(output_folder, "Raster_Outputs")

# Print information to screen for reference
print('Command to run:\n')
print('python Examine_Outputs_of_GIS_Preprocessor.py \\\n\t -i {0} \\\n\t -o {1}\n'.format(out_zip, raster_outputs))

# Run the script with required parameters
! python Examine_Outputs_of_GIS_Preprocessor.py -i {out_zip} -o {raster_outputs}

### Visualize the hydrologic routing grids

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

def see_raster(x):
    src = rasterio.open(os.path.join(raster_outputs, f"{x}.tif"))
    cmap, norm = cmap_options(x)
    if x in ['TOPOGRAPHY']:
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', norm=norm, interpolation='nearest', vmin=0)
    else:
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', norm=norm, interpolation='nearest')
    cbar = pyplot.colorbar()

    # Keep the automatic aspect while scaling the image up in size
    fig = pyplot.gcf()
    w, h = fig.get_size_inches()
    fig.set_size_inches(w * 1.75, h * 1.75)

    # Show image
    pyplot.show()

in_raster = widgets.Dropdown(
    options=[('Basin', 'BASIN'), ('Basin mask', 'basn_msk'), ('Channel grid', 'CHANNELGRID'), ('Flow accumulation', 'FLOWACC'),
            ('Flow direction', 'FLOWDIRECTION'), ('Forecast points', 'frxst_pts'), ('Lake grid', 'LAKEGRID'),
            ('Land use', 'landuse'), ('Latitude', 'LATITUDE'), ('LKSATFAC', 'LKSATFAC'), ('Longitude', 'LONGITUDE'),
            ('OVROUGHRTFAC', 'OVROUGHRTFAC'), ('RETDEPRTFAC', 'RETDEPRTFAC'), ('Stream order', 'STREAMORDER'),
            ('Topography', 'TOPOGRAPHY')],
    value='FLOWACC',
    description='Variable:')

interact(see_raster, x=in_raster)